<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_02_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import os

In [3]:
assert ('google.colab' in sys.modules), "ERROR - the script expects to be run in Colab"

In [4]:
# we persist code, dataset and runs on google drive...
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
# we probably don't want to do a "git fetch --dry-run here... the folders and
# versions are already set from the prior script colab_01 ..."
# so even if they are out-of-date we wouldn't want to update them here, we'd go
# back to the earlier script...

work_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/'
os.chdir(work_folder)
print("checking git status (git fetch --dry-run):")
!pwd
!git fetch --dry-run

checking git status (git fetch --dry-run):
/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 589 bytes | 15.00 KiB/s, done.
From https://github.com/cwinsor/medical_image_uw_madison
   51d8273..530ff1e  main       -> origin/main


# Install dependencies

In [10]:
pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
run_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT'
os.chdir(run_folder)
!pwd
!pip install -r ../requirements_cw.txt  # NOTE this is "cw" requirements one level up

/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT
  Using cached albumentations-1.0.3-py3-none-any.whl (98 kB)
  Using cached einops-0.3.2-py3-none-any.whl (25 kB)
  Using cached ignite-1.1.0-py2.py3-none-any.whl (4.5 kB)
  Using cached mmcv-1.3.17.tar.gz (390 kB)
  Preparing metadata (setup.py) ... done
  Using cached mmcv-full-1.3.17.tar.gz (390 kB)
  Preparing metadata (setup.py) ... done
  Using cached nibabel-3.2.2-py3-none-any.whl (3.3 MB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.9 MB)
  Using cached pandas-1.2.4.tar.gz (5.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Pillow-9.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached pretrainedmodels-0.7.4.tar.gz (58 kB)
  Preparing metadata (setup.py) ... done
  Using cached prettytable-2.2.0-py3-n

# Add to PATH (if required)

## Comparing ***os.getenv('PATH')*** to ***sys.path***
* the former is path for the environment shell (applications)
* the latter is the python search path for modules. Importantly it is constructed when Python *IS STARTED* and is:
  * the folder of the python interpreter
  * installation-dependent standard folders
  * the contents of PYTHONPATH
  
in other words... updating PYTHONPATH won't help inside a jupyter notebook!


In [29]:
THE_FOLDER = "foo"
# THE_FOLDER = os.getcwd()

if THE_FOLDER in sys.path:
    print("already exists")
else:
    sys.path.append(THE_FOLDER)
    print("added")

already exists


# PyTorch Distributed...

https://pytorch.org/docs/stable/distributed.html

    How to use this module:

    Single-Node multi-process distributed training

    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3 and all other
            arguments of your training script)

    Multi-Node multi-process distributed training: (e.g. two nodes)

    Node 1: (IP: 192.168.1.1, and has a free port: 1234)
    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            --nnodes=2 --node-rank=0 --master-addr="192.168.1.1"
            --master-port=1234 YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3
            and all other arguments of your training script)

    Node 2:
    python -m torch.distributed.launch --nproc-per-node=NUM_GPUS_YOU_HAVE
            --nnodes=2 --node-rank=1 --master-addr="192.168.1.1"
            --master-port=1234 YOUR_TRAINING_SCRIPT.py (--arg1 --arg2 --arg3
            and all other arguments of your training script)


# Ours is single-node multi-GPU.
So the shell command would be:

    cd /content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UMWGIT

    CONFIG='work_configs/tract/final_solution/classification_configs/cls_1.py'
    NUM_GPUS_PER_NODE=2
    MASTER_PORT=1234
    ARGS='foo="bar" lala="foofoo"'

    python -m torch.distributed.launch --nproc-per-node=$NUM_GPUS_PER_NODE
            tools/train.py $CONFIG --launcher pytorch $ARGS
            

# let's do that now...

In [46]:
# !pip install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.5/469.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 21.9 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-2.0.1-cp310-cp310-linux_x86_64.whl size=27640938 sha256=38c8eea51f5d731d3890930ed4228e647c376f04dcda326318fee29d8d84ba9e
  Stored in directory: /root/.cache/pip/wheels/17/58/eb/d8f23c5eb52bc7441f5b41a682a03b35277fc8cd4779d200b9
Successfully built mmcv
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 4.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is lik

In [53]:
pip list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
addict                           2.4.0
aiohttp                          3.8.4
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.2.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      21.3.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.0
arviz                            0.15.1
astropy                          5.2.2
astunparse                       1.6.3
async-timeout                    4.0.2
attrs                            23.1.0
audioread                        3.0.0
autograd                         1.6.2
Babel                            2.12.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bleach                           6.0.0


In [47]:
run_folder = '/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT'
os.chdir(run_folder)
!pwd

/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT


In [48]:
CONFIG='work_configs/tract/final_solution/classification_configs/cls_1.py'
NUM_GPUS_PER_NODE=1
MASTER_PORT=1234
ARGS='foo="bar" lala="foofoo"'

In [49]:
!python -m torch.distributed.launch --nproc-per-node=$NUM_GPUS_PER_NODE tools/train.py $CONFIG --launcher pytorch $ARGS

/usr/local/lib/python3.10/dist-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Colab_UW_Madison/medical_image_uw_madison/project/Kaggle-UWMGIT/tools/train.py", line 11, in <module>
    from mmcv.cnn.utils import revert_sync_batchnorm
ImportError: cannot import name 'revert_sync_batchnorm' from 'mmcv.cnn.utils' (/usr/local/lib/python3.10/dist-packages/mmcv/cnn/utils/__init__.py)
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 1) local_rank: 0 (pid: 65446) of binary: /usr/bin/python3
Traceback (most recent call last